## 1. Inicializar Spark y crear el contexto

En esta sección:
- Importamos las librerías necesarias.
- Creamos una `SparkSession`.
- Obtenemos el `SparkContext` para trabajar con RDD.

In [1]:
# 1. Importar librerías y crear SparkSession

from pyspark.sql import SparkSession
import os, sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# Creamos/obtenemos una sesión de Spark
# spark = SparkSession.builder \
#     .appName("AnalisisDiscursoPresidencial") \
#     .getOrCreate()

spark = (
    SparkSession.builder
    .appName("AnalisisDiscursoPresidencial")
    .config("spark.pyspark.python", sys.executable)
    .config("spark.pyspark.driver.python", sys.executable)
    .getOrCreate()
)

# Obtenemos el SparkContext para trabajar con RDD
sc = spark.sparkContext

print("Versión de Spark:", spark.version)

Versión de Spark: 4.0.2


## 2. Cargar el archivo de texto como RDD

En esta sección:
- Definimos la ruta del archivo `05 - discurso.txt`.
- Cargamos el archivo usando `sc.textFile`.
- Vemos algunas líneas para verificar la lectura.

In [ ]:
# 2. Cargar el archivo de discurso como RDD

# Ruta del archivo de texto.
# Si el notebook está en la misma carpeta que el archivo (M8/S2),
# esta ruta relativa debería funcionar directamente.
file_path = "/content/05 - discurso.txt"

# Cargamos el archivo como un RDD de líneas
lines_rdd = sc.textFile(file_path)


# Mostramos algunas líneas para verificar el contenido
print("Número aproximado de líneas:", lines_rdd.count())
print("\nPrimeras líneas del discurso:\n")
for line in lines_rdd.take(5):
    print(line)

Número aproximado de líneas: 19

Primeras líneas del discurso:

Queridos compatriotas chilenos,

Hoy me dirijo a ustedes con un corazón lleno de esperanza y determinación, consciente de los desafíos que enfrentamos como nación, pero también confiada en nuestra capacidad para superarlos juntos.

En estos tiempos de incertidumbre y cambios rápidos, es crucial que permanezcamos unidos como pueblo. La unidad es nuestra mayor fortaleza y nuestra mejor defensa contra las adversidades que puedan surgir en nuestro camino.


## 3. Limpieza de texto y definición de stopwords

En esta sección:
- Pasamos el texto a minúsculas.
- Eliminamos signos de puntuación y caracteres no alfabéticos.
- Separamos el texto en palabras.
- Definimos una lista de **stopwords** (palabras sin mucho significado para el análisis: artículos, pronombres, etc.).

In [3]:
# 3. Funciones de limpieza y lista de stopwords

import re

# Lista de palabras vacías (stopwords) en español.
# Se puede ampliar según necesidad.
stopwords = {
    "a", "e", "i", "o", "u",
    "el", "la", "los", "las", "un", "una", "unos", "unas",
    "y", "o", "u", "de", "del", "al", "en", "con", "por", "para",
    "como", "que", "se", "su", "sus", "es", "son", "ser", "fue", "fueron",
    "este", "esta", "estos", "estas", "ese", "esa", "esos", "esas",
    "yo", "tú", "tu", "usted", "ustedes", "nosotros", "nosotras",
    "ellos", "ellas",
    "mi", "mis", "nuestro", "nuestra", "nuestros", "nuestras",
    "su", "sus",
    "un", "una", "al", "del",
    "pero", "también", "ya", "muy", "más", "menos",
    "que", "porque", "cuando", "donde",
    "hoy", "ayer", "mañana",
    "es", "son", "era", "eran", "ser", "estar", "está", "están"
}

def limpiar_y_dividir(linea):
    """Convierte la línea a minúsculas, elimina puntuación y la separa en palabras."""
    # Pasamos a minúsculas
    linea = linea.lower()
    # Reemplazamos cualquier caracter que no sea letra (incluye tildes y ñ) por un espacio
    # Conservamos letras a-z, áéíóúñü
    linea = re.sub(r"[^a-záéíóúñü]+", " ", linea)
    # Dividimos en palabras por espacios
    palabras = linea.split()
    return palabras

# Probamos la función de limpieza con una línea de ejemplo
ejemplo = "¡Queridos compatriotas chilenos! Hoy me dirijo a ustedes..."
print("Línea original:", ejemplo)
print("Palabras limpias:", limpiar_y_dividir(ejemplo))

Línea original: ¡Queridos compatriotas chilenos! Hoy me dirijo a ustedes...
Palabras limpias: ['queridos', 'compatriotas', 'chilenos', 'hoy', 'me', 'dirijo', 'a', 'ustedes']


## 4. Pipeline con RDD: contar palabras

Pasos con RDD:
1. `flatMap` para transformar líneas en palabras.
2. `filter` para eliminar palabras vacías (stopwords) y cadenas vacías.
3. `map` para transformar cada palabra en un par `(palabra, 1)`.
4. `reduceByKey` para sumar las ocurrencias por palabra.
5. `sortBy` para ordenar por frecuencia de mayor a menor.
6. `take(10)` para obtener las **10 palabras más frecuentes**.

In [4]:
# 4. Construir el pipeline RDD para conteo de palabras

# 4.1. Pasar de líneas a palabras limpias usando flatMap
palabras_rdd = lines_rdd.flatMap(limpiar_y_dividir)

# 4.2. Filtrar stopwords y palabras vacías
palabras_filtradas_rdd = palabras_rdd.filter(
    lambda w: w and (w not in stopwords)
)

# 4.3. Mapear cada palabra a un par (palabra, 1)
pares_rdd = palabras_filtradas_rdd.map(lambda w: (w, 1))

# 4.4. Reducir por clave (palabra) sumando las ocurrencias
conteo_rdd = pares_rdd.reduceByKey(lambda a, b: a + b)

# 4.5. Ordenar por frecuencia (valor) de mayor a menor
conteo_ordenado_rdd = conteo_rdd.sortBy(lambda par: par[1], ascending=False)

# 4.6. Tomar las 10 palabras más frecuentes (acción RDD)
top_10_palabras = conteo_ordenado_rdd.take(10)

top_10_palabras

[('nos', 3),
 ('juntos', 3),
 ('me', 2),
 ('enfrentamos', 2),
 ('tiempos', 2),
 ('mejor', 2),
 ('presidenta', 2),
 ('compromiso', 2),
 ('cada', 2),
 ('construir', 2)]

## 5. Mostrar los 10 principales términos formateados

En esta sección simplemente mostramos los resultados de forma más amigable: palabra y cantidad de apariciones.

In [5]:
# 5. Imprimir los 10 términos más frecuentes de forma legible

print("Top 10 palabras más frecuentes (sin stopwords):\n")
for palabra, frecuencia in top_10_palabras:
    print(f"{palabra:15s} -> {frecuencia}")

Top 10 palabras más frecuentes (sin stopwords):

nos             -> 3
juntos          -> 3
me              -> 2
enfrentamos     -> 2
tiempos         -> 2
mejor           -> 2
presidenta      -> 2
compromiso      -> 2
cada            -> 2
construir       -> 2


## 6. Cierre de la sesión de Spark

Al finalizar el análisis, es buena práctica detener la sesión de Spark.

In [6]:
# 6. Detener la sesión de Spark

spark.stop()